In [1]:
import ccxt
import pandas as pd
from datetime import datetime
# pip install finance-datareader 패키지 이름 주의
import FinanceDataReader as fdr

In [2]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds() # 현재시간?
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe) # 초단위로 바꿈
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    if isinstance(since, str):
        since = exchange.parse8601(since)

    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        if len(all_ohlcv) % 100000 == 0:
            print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [3]:
binance = ccxt.binance()
all_ohlcv = scrape_ohlcv(binance, 3, 'XRP/USDT', '1m', '2017-01-01 00:00:00Z', 1000)

100000 XRP/USDT candles in total from 2021-09-16T02:02:00.000Z to 2021-11-24T12:41:00.000Z
200000 XRP/USDT candles in total from 2021-07-08T15:22:00.000Z to 2021-11-24T12:41:00.000Z
300000 XRP/USDT candles in total from 2021-04-30T04:42:00.000Z to 2021-11-24T12:41:00.000Z
400000 XRP/USDT candles in total from 2021-02-19T18:45:00.000Z to 2021-11-24T12:41:00.000Z
500000 XRP/USDT candles in total from 2020-12-12T08:05:00.000Z to 2021-11-24T12:41:00.000Z
600000 XRP/USDT candles in total from 2020-10-03T21:25:00.000Z to 2021-11-24T12:41:00.000Z
700000 XRP/USDT candles in total from 2020-07-26T10:45:00.000Z to 2021-11-24T12:41:00.000Z
800000 XRP/USDT candles in total from 2020-05-18T00:05:00.000Z to 2021-11-24T12:41:00.000Z
900000 XRP/USDT candles in total from 2020-03-09T15:50:00.000Z to 2021-11-24T12:41:00.000Z
1000000 XRP/USDT candles in total from 2019-12-31T05:10:00.000Z to 2021-11-24T12:41:00.000Z
1100000 XRP/USDT candles in total from 2019-10-22T20:00:00.000Z to 2021-11-24T12:41:00.00

In [4]:
dat = pd.DataFrame(all_ohlcv, columns=['time','open','high','low','close','volume'])
dat['time'] = [datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S') for t in dat['time']]
dat.to_csv('XRP-USDT_1min.csv', index=False)

환율데이터 불러와서 저장

In [22]:
# pip install finance-datareader 패키지 이름 주의

import FinanceDataReader as fdr


ex_rate = fdr.DataReader('USD/KRW','2018')
ex_rate = ex_rate.reset_index()[['Date','Close']]
ex_rate['Date'] = ex_rate['Date'].astype('str')
ex_rate.head()

# index 사용해서 전처리가 용이
ex_rate = ex_rate.set_index(['Date'])
ex_rate.index = pd.DatetimeIndex(ex_rate.index)

# 공휴일 빈날짜까지 다 채워서 환율 만들기
idx = pd.date_range(ex_rate.index[0],ex_rate.index[-1])
ex_rate = ex_rate.reindex(idx)

# 금요일값으로 토일 채우기 : 밀어채우기
ex_rate = ex_rate.fillna(method='pad')
ex_rate = ex_rate.reset_index().rename(columns={"index": "Date"})
ex_rate.to_csv('USD-KRW_2018-recent.csv', index=False)


In [28]:
dat = pd.read_csv('XRP-USDT_1min.csv')
dat['time'] = pd.to_datetime(dat['time'])
# yyyy-mm-dd 형식 변수 만들어주기
dat['ymd'] = dat['time'].dt.date.astype('str')

ex_rate = pd.read_csv('USD-KRW_2018-recent.csv')

dfs = pd.DataFrame()
for df in dat.groupby(['ymd']):
    # df[0]:날짜, df[1]:dateframe
    USD = ex_rate.loc[ex_rate['Date']==df[0]]['Close']
    df[1].loc[:, ['open','high','low','close']] *= float(USD)
    dfs = dfs.append(df[1])
    # print(USD)
dfs[1:].to_csv('XRP-USDT_1min.csv', index=False)

In [29]:
dat = pd.read_csv('XRP-USDT_1min.csv')

In [30]:
dat

,time,open,high,low,close,volume,ymd
0,2018-05-04 17:12:00,966.699000,998.911559,912.993500,966.699000,167543.47,2018-05-04
1,2018-05-04 17:13:00,966.699000,998.922300,955.968641,977.440100,179750.59,2018-05-04
2,2018-05-04 17:14:00,977.440100,1611.165000,966.795670,998.911559,107222.48,2018-05-04
3,2018-05-04 17:15:00,998.911559,1020.415241,977.654922,977.654922,171304.56,2018-05-04
4,2018-05-04 17:16:00,977.654922,998.718219,977.654922,998.675255,62126.16,2018-05-04
...,...,...,...,...,...,...,...
1871994,2021-11-23 21:29:00,1233.908316,1234.621764,1233.670500,1234.146132,246768.00,2021-11-23
1871995,2021-11-23 21:30:00,1234.146132,1234.146132,1231.292340,1231.530156,142052.00,2021-11-23
1871996,2021-11-23 21:31:00,1231.411248,1231.886880,1229.865444,1231.767972,582344.00,2021-11-23
1871997,2021-11-23 21:32:00,1231.767972,1231.886880,1230.341076,1230.341076,324938.00,2021-11-23


In [34]:
def make_candle(df,interval = 10, open_time: str = None ):
    from datetime import datetime, timedelta
    offset = timedelta(minutes = interval)
    if open_time is None:
        start_date = df.index[0]
    else: 
        start_date = df.index[0].replace(hour = int(open_time[:2]), minute = int(open_time[3:]))
    end_date = df.index[-1]
    counts = (pd.to_datetime(end_date)-pd.to_datetime(start_date)) // offset + 1
    rows = []
    for i in range(counts):
        reduced_df = df.loc[start_date + i * offset : start_date + (i+1)*offset - timedelta(minutes = 1)]
        try :#예전자료에는 없는 분봉들이 있음
            date = start_date + i * offset
            open = reduced_df['open'][0]
            high = reduced_df['high'].max()
            low = reduced_df['low'].min()
            close = reduced_df['close'][-1]
            volume = reduced_df['volume'].sum()
            rows.append((date, open, high,low,close,volume))
        except IndexError:
            continue
    data = pd.DataFrame(rows, columns = ['date', 'open', 'high','low','close', 'volume'])
    data.index = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M')
    data = data.drop("date", axis =1 )
    return data

In [38]:
dat.set_index('time', inplace=True)

In [44]:
from datetime import datetime, timedelta
df=dat;interval = 60*24; open_time: str = None 

In [49]:
offset = timedelta(minutes = interval)
if open_time is None:
    start_date = df.index[0]
else: 
    start_date = df.index[0].replace(hour = int(open_time[:2]), minute = int(open_time[3:]))
end_date = df.index[-1]
counts = (pd.to_datetime(end_date)-pd.to_datetime(start_date)) // offset + 1
# rows = []
# for i in range(counts):
#     reduced_df = df.loc[start_date + i * offset : start_date + (i+1)*offset - timedelta(minutes = 1)]
#     try :#예전자료에는 없는 분봉들이 있음
#         date = start_date + i * offset
#         open = reduced_df['open'][0]
#         high = reduced_df['high'].max()
#         low = reduced_df['low'].min()
#         close = reduced_df['close'][-1]
#         volume = reduced_df['volume'].sum()
#         rows.append((date, open, high,low,close,volume))
#     except IndexError:
#         continue
# data = pd.DataFrame(rows, columns = ['date', 'open', 'high','low','close', 'volume'])
# data.index = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M')
# data = data.drop("date", axis =1 )

In [50]:
(pd.to_datetime(end_date)-pd.to_datetime(start_date)) // offset + 1

1300

In [52]:
make_candle(dat,interval = 60*24, open_time = None )

TypeError: can only concatenate str (not "datetime.timedelta") to str